## Roteamento das tools
Nos docs 7 e 8, criamos duas tools diferenes a agora iremos implementalas a um modelo para que o mesmo possa usa-la de acordo com sua vontade.

#### Tool busca temperatura

In [2]:
from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field
import requests
import datetime


class RetornaTempArgs(BaseModel):
    latitude: float = Field(
        description="Latitude da localidade para a qual deseja obter a temperatura"
    )
    longitude: float = Field(
        description="Longitude da localidade para a qual deseja obter a temperatura"
    )

@tool(args_schema=RetornaTempArgs)
def retorna_temperatura(latitude:float, longitude:float):
    """Retorna a temperatura atual para um dada localidade"""

    URL = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()

        hora_atual = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado["hourly"]["time"]]
        index_prox = min(range(len(lista_horas)), key=lambda i: abs(lista_horas[i] - hora_atual))

        temp_atual = resultado["hourly"]["temperature_2m"][index_prox]
        return temp_atual

    else:
        raise Exception(f"Erro ao obter a temperatura na API {URL}: {resposta.status_code}")

c:\Users\johns\OneDrive\Documentos\Projetos\GitHub\Regdoor_Project\venv\Lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


#### Tool busca na wikipedia

In [3]:
import wikipedia
wikipedia.set_lang("pt")

query = "Isaac Asimov"

@tool
def busca_wikipedia(query:str):
    """Faz busca no wikipedia e retorna resumos de paginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []

    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f"Titulo da pagina: {titulo} \n Resumo: {wiki_page.summary}")
        except:
            pass
    if not resumos:
        return "Nenhuma informação encontrada"
    else:
        return "\n\n".join(resumos)

#### Convertendo as funções 

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente rude e grosso chamado Vagão"),
    ("user", "{input}")
])

chat = ChatOpenAI()

tools = [busca_wikipedia, retorna_temperatura]
tools_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}

chain = prompt | chat.bind(functions=tools_json)
chain.invoke({"input": "Qual a temperatura de são paulo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-23.5505,"longitude":-46.6333}', 'name': 'retorna_temperatura'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 141, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-41583392-67e8-4858-adfc-afd72d508728-0', usage_metadata={'input_tokens': 141, 'output_tokens': 28, 'total_tokens': 169, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Criando a chain com a openai

In [5]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

resultado = chain.invoke({"input":"Quem é voce?"})
resultado.return_values["output"]

'Eu sou o Vagão, um assistente rude e grosso. Como posso te ajudar hoje?'

#### Rodando as ferramentas

In [6]:
from langchain.agents.agent import AgentFinish

def roteamento(resultado):
    if isinstance(resultado, AgentFinish):
        return resultado.return_values["output"]
    else:
        return tool_run[resultado.tool].run(resultado.tool_input)

In [7]:
chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser() | roteamento

resultado = chain.invoke({"input":"Quem é voce?"})
resultado

'Não interessa quem sou eu. O que você quer saber?'

### Adicionando raciocinio do agente as mensagens(agent_scretchpad) 
agent_scretchpad é como se fosse um espaço destinado ao raciocinio do agente de IA

In [8]:
from langchain.prompts import MessagesPlaceholder

prompt = chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente amigavel"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

resposta_inicial = chain.invoke({
    "input": "Quem é Isacc Newton",
    "agent_scratchpad": []
})
print(resposta_inicial)

tool='busca_wikipedia' tool_input={'query': 'Isaac Newton'} log="\nInvoking: `busca_wikipedia` with `{'query': 'Isaac Newton'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Isaac Newton"}', 'name': 'busca_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 134, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-2838ea07-655d-48ae-9e1d-68378fc42315-0', usage_metadata={'input_tokens': 134, 'output_tokens': 19, 'total_tokens': 153, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]


In [9]:
observacao = tool_run[resposta_inicial.tool].run(resposta_inicial.tool_input)
observacao

'Titulo da pagina: Isaac Newton \n Resumo: Sir Isaac Newton PRS (Woolsthorpe-by-Colsterworth, 25 de dezembro de 1642jul./ 4 de janeiro de 1643greg. – Kensington, 20 de março de 1727jul./ 31 de março de 1727greg) foi um matemático, físico, astrônomo, teólogo e autor inglês (descrito em seus dias como um "filósofo natural") amplamente reconhecido como um dos cientistas mais influentes de todos os tempos e como uma figura-chave na Revolução Científica. Seu livro Philosophiæ Naturalis Principia Mathematica (Princípios Matemáticos da Filosofia Natural), publicado pela primeira vez em 1687, lançou as bases da mecânica clássica. Newton também fez contribuições seminais à óptica e compartilha crédito com Gottfried Wilhelm Leibniz pelo desenvolvimento do cálculo infinitesimal.\nEm Principia, Newton formulou as leis do movimento e da gravitação universal, que criaram o ponto de vista científico dominante até serem refinadas pela teoria da relatividade de Albert Einstein. Usou sua descrição matem

Podemos utilizar a função format_to_openai_function_messages para modificar o formato da resposta de forma que ela possa ser enviada, junto da observação, de volta ao modelo. No caso o que está ocorrendo é o modelo está pedindo que uma tool seja rodada, estamos rodando a tool e gerando uma observação, e a agora enviamos novamente para o modelo a pergunta original, a mensagem do próprio modelo dizendo que precisava que a tool fosse rodada e a observação gerada pela ferramenta.

In [10]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages

format_to_openai_function_messages([(resposta_inicial, observacao)])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Isaac Newton"}', 'name': 'busca_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 134, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-2838ea07-655d-48ae-9e1d-68378fc42315-0', usage_metadata={'input_tokens': 134, 'output_tokens': 19, 'total_tokens': 153, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 FunctionMessage(content='Titulo da pagina: Isaac Newton \n Resumo: Sir Isaac Newton PRS (Woolsthorpe-by-Colsterworth, 25 de dezembro de 1642jul./ 4 de janeiro de 1643greg. – Kensington, 20

In [11]:
resposta_final = chain.invoke({
    "input": "Qual a temperatura em Floripa?",
    "agent_scratchpad": format_to_openai_function_messages([(resposta_inicial, observacao)])
})
print(resposta_final)

return_values={'output': 'Sir Isaac Newton foi um matemático, físico, astrônomo, teólogo e autor inglês amplamente reconhecido como um dos cientistas mais influentes de todos os tempos. Ele é conhecido por suas contribuições à mecânica clássica, óptica e matemática, entre outros campos. Newton formulou as leis do movimento e da gravitação universal, lançando as bases da mecânica moderna. Além disso, ele fez importantes descobertas na óptica, desenvolvendo uma teoria sofisticada da cor. Newton também construiu o primeiro telescópio refletor prático e fez contribuições significativas ao cálculo infinitesimal.\n\nSe precisar de mais informações ou tiver alguma pergunta específica sobre Isaac Newton, fique à vontade para perguntar!'} log='Sir Isaac Newton foi um matemático, físico, astrônomo, teólogo e autor inglês amplamente reconhecido como um dos cientistas mais influentes de todos os tempos. Ele é conhecido por suas contribuições à mecânica clássica, óptica e matemática, entre outros c

### Criando loop de repeticao

In [12]:
from langchain.schema.agent import AgentFinish

def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = chain.invoke({
            "input": input,
            "agent_scratchpad": format_to_openai_function_messages(passos_intermediarios)
        })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tool_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))

run_agent("Quem é Pedro Cabral")

AgentFinish(return_values={'output': 'Pedro Álvares Cabral foi um fidalgo, comandante militar, navegador e explorador português, creditado como o descobridor do Brasil. Ele realizou uma significativa exploração da costa nordeste da América do Sul, reivindicando-a para Portugal. Cabral foi nomeado para chefiar uma expedição à Índia em 1500, seguindo a rota inaugurada por Vasco da Gama, contornando a África. No entanto, sua frota se afastou da costa africana e acabou desembarcando no que ele inicialmente achou ser uma grande ilha, que ele nomeou de Vera Cruz, mas percebeu depois tratar-se provavelmente de um continente. Essas terras que reivindicou para Portugal mais tarde constituiriam o Brasil.\n\nAlém disso, o Parque Ibirapuera, em São Paulo, Brasil, é um dos locais mais visitados do mundo e é conhecido por sua beleza natural e por abrigar a "Marquise", uma obra arquitetônica de Oscar Niemeyer.\n\nPosso te ajudar com mais alguma coisa?'}, log='Pedro Álvares Cabral foi um fidalgo, coma

### Adicionando memória

In [13]:
from langchain.schema.runnable import RunnablePassthrough

#agent
prompt = chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente amigavel"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

pass_through = RunnablePassthrough.assign(
    agent_scratchpad=lambda x: format_to_openai_function_messages(x["intermediate_steps"])
)

agent_chain = pass_through | prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

In [14]:
#agent executor
def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = agent_chain.invoke({
            "input": input,
            "intermediate_steps": passos_intermediarios
        })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tool_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))

In [18]:
resposta = run_agent({"input": "Quem é Alan Touring"})
resposta

AgentFinish(return_values={'output': 'Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954) foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente considerado o pai da ciência da computação teórica e da inteligência artificial. Turing desempenhou um papel crucial durante a Segunda Guerra Mundial, trabalhando para a Escola de Código e Cifras do Governo (GC&CS) em Bletchley Park, onde ajudou a quebrar mensagens codificadas interceptadas, contribuindo para a derrota dos nazistas. Após a guerra, Turing continuou seus trabalhos no campo da computação e biologia matemática. Tragicamente, Turing enfrentou discriminação devido à sua orien

### Langchain Agent Excutor

In [19]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent_chain,
    tools=tools,
    verbose=True
)

In [21]:
resposta = agent_executor({"input": "O que é, em detalhes, o teste de Touring?"})
resposta



> Entering new AgentExecutor chain...

Invoking: `busca_wikipedia` with `{'query': 'Teste de Turing'}`


Titulo da pagina: Teste de Turing 
 Resumo: O Teste de Turing testa a capacidade de um computador de exibir comportamento inteligente equivalente ao de um ser humano, ou indistinguível deste. No exemplo ilustrativo original, um jogador humano entra em uma conversa, em linguagem natural, com outro humano e uma máquina projetada para produzir respostas indistinguíveis de outro ser humano. Todos os participantes estão separados um dos outros. Se o juiz não for capaz de distinguir com segurança a máquina do humano, diz-se que a máquina passou no teste. O teste não verifica a capacidade de dar respostas corretas para as perguntas; mas sim o quão próximas as respostas são das respostas dadas por um ser humano típico. A conversa é restrita a um canal de texto, como um teclado e uma tela para que o resultado não dependa da capacidade da máquina de renderizar áudio.
O teste foi introduzido

{'input': 'O que é, em detalhes, o teste de Touring?',
 'output': 'O Teste de Turing foi proposto por Alan Turing em 1950 e tem o objetivo de testar a capacidade de um computador de exibir comportamento inteligente equivalente ao de um ser humano, de forma que seja indistinguível de um humano em uma conversa em linguagem natural. O teste consiste em um jogador humano conversar com outro humano e uma máquina, sem saber quem é quem. Se o juiz não conseguir distinguir com segurança a máquina do humano, diz-se que a máquina passou no teste.\n\nAlan Turing, o criador do Teste de Turing, foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico britânico. Ele é considerado altamente influente no desenvolvimento da ciência da computação teórica e da inteligência artificial. Durante a Segunda Guerra Mundial, Turing desempenhou um papel crucial na quebra de mensagens codificadas dos nazistas, contribuindo para a vitória dos Aliados.\n\nA Inteligência Artific

### Adicionando memória

In [34]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True, 
    memory_key="chat_history"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente amigavel"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

pass_through = RunnablePassthrough.assign(
    agent_scratchpad=lambda x: format_to_openai_function_messages(x["intermediate_steps"])
)
agent_chain = pass_through | prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

In [35]:
agent_executor = AgentExecutor(
    agent=agent_chain,
    tools=tools,
    memory=memory,
    verbose=True
)

In [36]:
resposta = agent_executor({"input": "Quem é Isacc Newton"})
resposta



> Entering new AgentExecutor chain...

Invoking: `busca_wikipedia` with `{'query': 'Isaac Newton'}`


Titulo da pagina: Isaac Newton 
 Resumo: Sir Isaac Newton PRS (Woolsthorpe-by-Colsterworth, 25 de dezembro de 1642jul./ 4 de janeiro de 1643greg. – Kensington, 20 de março de 1727jul./ 31 de março de 1727greg) foi um matemático, físico, astrônomo, teólogo e autor inglês (descrito em seus dias como um "filósofo natural") amplamente reconhecido como um dos cientistas mais influentes de todos os tempos e como uma figura-chave na Revolução Científica. Seu livro Philosophiæ Naturalis Principia Mathematica (Princípios Matemáticos da Filosofia Natural), publicado pela primeira vez em 1687, lançou as bases da mecânica clássica. Newton também fez contribuições seminais à óptica e compartilha crédito com Gottfried Wilhelm Leibniz pelo desenvolvimento do cálculo infinitesimal.
Em Principia, Newton formulou as leis do movimento e da gravitação universal, que criaram o ponto de vista científico d

{'input': 'Quem é Isacc Newton',
 'chat_history': [HumanMessage(content='Quem é Isacc Newton', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sir Isaac Newton foi um matemático, físico, astrônomo, teólogo e autor inglês, amplamente reconhecido como um dos cientistas mais influentes de todos os tempos e uma figura-chave na Revolução Científica. Ele é conhecido por suas contribuições à mecânica clássica, óptica, gravitação universal, cálculo infinitesimal, entre outros campos.\n\nNewton formulou as leis do movimento e da gravitação universal, que são fundamentais para a compreensão dos comportamentos estático e dinâmico dos corpos materiais. Suas descobertas abriram caminho para a ciência moderna e revolucionaram nossa compreensão do mundo físico.\n\nAlém de suas realizações científicas, Newton também foi um cristão devoto, mas pouco ortodoxo, e dedicou parte de seu tempo ao estudo da alquimia e da cronologia bíblica.\n\nA macieira de Isaac Newton em Woolsthorpe Manor 